# Real world example: Maintenance scripts

## Scenario
In your OpenBIS instance exists a number of objects of type CHEMICAL. We create maintenance scripts to perform the following actions:
* Check properties: checks the expiration date and saves a list of expired chemicals in a custom format.
* Storage change: changes the location of some chemicals to another room.

This example shows the **interactive development process** - step by step from the first line to the complete scripts.

**As all users work on the same instance, we will not modify the global INVENTORY but limit the actions to the user's space. But this code may be used for the inventory as well, just change the SPACE limitation condition.**

## Start: connecting to openBIS
Using URL and username+password or PAT

In [ ]:
from pybis import Openbis
o = Openbis('https://schulung.datastore.bam.de')
o.login('mmusterm', 'bamisgreat')

## OPTIONAL: create example data for this example
This example needs some objects of type CHEMICAL to work. If you don't have example objects to play with you may create some with the following code.

In [ ]:
SPACE='MMUSTERM'
PROJECT='PYBISTUTORIAL'
COLLECTION='MYCHEMICALS'

space = o.get_space(SPACE)
try:
    proj = space.get_project(PROJECT)
except ValueError:
    proj=o.new_project(space=SPACE, code=PROJECT, description='just for learning pyBIS')
    proj.save()
try:
    coll = space.get_collection(COLLECTION)
except ValueError:
    coll=o.new_collection(project=proj, code=COLLECTION, type='COLLECTION')
    coll.save()

# FIXME: do a loop and create some more chemicals!
c = o.new_object(type='CHEMICAL', space=space, collection=coll, props={
    '$name': 'Water',
    'description': 'just pure water',
    'manufacturer': 'ACME Corp.',
    'hazardous_substance': False,
    'bam_oe': 'OE_S.3',
    'bam_location_complete': 'UE_02_2_310',
    'date_expiration': '2023-12-31'
})
c.save()
c = o.new_object(type='CHEMICAL', space=space, collection=coll, props={
    '$name': 'Ethanol',
    'description': 'hicks!',
    'manufacturer': 'ACME Corp.',
    'hazardous_substance': True,
    'bam_oe': 'OE_S.3',
    'bam_location_complete': 'UE_02_2_310'
})
c.save()
c = o.new_object(type='CHEMICAL', space=space, collection=coll, props={
    '$name': 'Propanol',
    'description': 'bäh!',
    'manufacturer': 'ACME Corp.',
    'hazardous_substance': True,
    'bam_oe': 'OE_S.3',
    'bam_location_complete': 'UE_02_2_309'
})
c.save()

## Example: Check for expiry

Check for expired CHEMICALs and export the list to a CSV file in a custom format.

### Searching all objects of type CHEMICAL

In [ ]:
o.get_objects(type='CHEMICAL')

### Limiting search to a SPACE

In [ ]:
SPACE='MMUSTERM'
o.get_objects(type='CHEMICAL', space=SPACE)

### Include needed properties in result

In [ ]:
o.get_objects(type='CHEMICAL', space=SPACE, props=['$name', 'date_expiration'])

### Check for expiration date

In [ ]:
ac = o.get_objects(type='CHEMICAL', space=SPACE, props=['$name', 'date_expiration'])
today = '2024-01-20'
expired = []
for c in ac:
    if c.props['date_expiration'] < today:
        print(c.code)
        expired.append(c)

You will run into trouble if there are chemicals without the expiration date (which is not mandatory). A value of `None` can't be compared to a string!

In [ ]:
expired = []
for c in ac:
    if c.props['date_expiration'] is not None and c.props['date_expiration'] < today:
        print(c.code)
        expired.append(c)

**Hint:** getting a value twice with `[]` may lead to performance issues. There are several workarounds:
```python
ed = c.props['date_expiration']
if ed is not None and ed < today:
    ...
```
or:
```python
if (c.props['date_expiration'] or '9999-12-31') < today:
    ...
```
The explicit `for`-loop can also be replaced with a shorter list comprehension, which lead to very compact code:

In [ ]:
expired = [c for c in ac if (c.props['date_expiration'] or '9999-12-31') < today]

### Use current date for comparison instead of hardcoded
A script should not contain a manually entered date - just get the current date.

In [ ]:
from datetime import date
today = date.today().isoformat()
expired = [c for c in ac if c.props['date_expiration'] is not None and c.props['date_expiration'] < today]
for c in expired:
    print(c.code)

### Save this list in a CSV file
Now the list shoulb be saved in afile that can be given to the colleagues maintaining the chemicals. A custom column layout should be used.

In [ ]:
import csv
fields = ('$name', 'hazardous_substance', 'description', 'bam_location_complete', 'date_expiration')
with open('EXPIRED_CHEMICALS.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    for c in expired:
        csvwriter.writerow([c.props[f] for f in fields])

### Putting it all together - the complete script
This is a combination of the above code in one script. All import statements were moved to the top.

In [ ]:
from pybis import Openbis
from datetime import date
import csv

SPACE='MMUSTERM'
fields = ('$name', 'hazardous_substance', 'description', 'bam_location_complete', 'date_expiration')

## connect and login - you should use a PAT instead
o = Openbis('https://schulung.datastore.bam.de')
o.login('mmusterm', 'bamisgreat')

# get all chemicals in space, include needed properties
ac = o.get_objects(type='CHEMICAL', space=SPACE, props=fields)

# filter out expired chemicals
from datetime import date
today = date.today().isoformat()
expired = [c for c in ac if (c.props['date_expiration'] or '9999-12-31') < today]

# write list as csv file
fields = ('$name', 'hazardous_substance', 'description', 'bam_location_complete', 'date_expiration')
with open('EXPIRED_CHEMICALS.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    for c in expired:
        csvwriter.writerow([c.props[f] for f in fields])

o.logout()

## Example: Room number change
A storage for chemicals has moved from one room to another. The location of all CHEMICALs with the old room number need to be changed to a new room number.

### Get rooms from BAM_LOCATION_COMPLETE

To change the room, we do not just need the label of the vocabulary term, but the CODE (internal name) of the entry. We could also get this via pyBIS, but this is not usable in current pyBIS versions. Please use the Vocabulare Browser in the ELN to get the codes.

In [ ]:
OLD_ROOMCODE = 'UE_02_2_310' # 'UE/02/2/310'
NEW_ROOMCODE = 'UE_02_2_309' # 'UE/02/2/309'

### Get list of CHEMICALs with old room number
This is almost the same code as in the previous example. The only new thing is the `where=` statement, which returns just chemicals with the property `bam_location_complete` matching the room.

In [ ]:
SPACE='MMUSTERM'
cc = o.get_objects(type='CHEMICAL', space=SPACE, props=['$name', 'bam_location_complete'], where={'bam_location_complete': OLD_ROOMCODE})
cc

### Iterate over list and set new room number

Change every single object in list and save it. This works well for a small number of objects.

In [ ]:
for c in cc:
    c.props['bam_location_complete'] = NEW_ROOMCODE
    c.save()

### Like before, but with transactions

In [ ]:
trans = o.new_transaction()
for c in cc:
    c.props['bam_location_complete'] = NEW_ROOMCODE
    trans.add(c)
trans.commit()

### Putting it all together - the complete script
This is a combination of the above code in one script. Do not forget to adjust the room numbers every time you run it.

In [ ]:
from pybis import Openbis

SPACE='MMUSTERM'
OLD_ROOMCODE = 'UE_02_2_310' # 'UE/02/2/310'
NEW_ROOMCODE = 'UE_02_2_309' # 'UE/02/2/305'

## connect and login - you should use a PAT instead
o = Openbis('https://schulung.datastore.bam.de')
o.login('mmusterm', 'bamisgreat')

# get all chemicals in space that have the old location
cc = o.get_objects(type='CHEMICAL', space=SPACE, props=['$name', 'bam_location_complete'], where={'bam_location_complete': OLD_ROOMCODE})

# create and execute transaction to change objects to new location in just one action on the server
trans = o.new_transaction()
for c in cc:
    c.props['bam_location_complete'] = NEW_ROOMCODE
    trans.add(c)
trans.commit()

o.logout()

## Logout

In [ ]:
o.logout()